# SAGA Wind Effect – Simple UI + Map Previews
Two cells only:
1. **Config & helpers** (defaults + functions).
2. **UI + Run button** – pick DEM, set direction, click **Run Wind Effect**.  
   Shows a small status message while processing and quick-look **maps** of the **input** and the **output**.

> Notes: Requires SAGA (`saga_cmd`), plus Python packages: `rasterio`, `numpy`, and `matplotlib` (for the previews).


In [ ]:
# --- Cell 1: Config & helper functions ---
import os, sys, shutil, subprocess
from pathlib import Path
from IPython.display import display
import ipywidgets as widgets
from ipyfilechooser import FileChooser

# === Defaults (edit as needed) ===
DEFAULT_MAXDIST_KM = 30.0
DEFAULT_ACCEL = 1.5
DEFAULT_PYRAMIDS = True
DEFAULT_OLDVER = False
DEFAULT_EXPORT_TIF = True

def _detect_saga_cmd():
    exe = 'saga_cmd.exe' if os.name == 'nt' else 'saga_cmd'
    found = shutil.which(exe)
    return found or ''

def _clean_path(p: str) -> str:
    p = (p or '').strip()
    if (p.startswith('"') and p.endswith('"')) or (p.startswith("'") and p.endswith("'")):
        p = p[1:-1]
    return os.path.expandvars(os.path.expanduser(p))

def _ensure_saga_exe(p: str) -> str:
    """Return a usable saga_cmd path or '' if not found. Accepts a file path or a folder path."""
    p = _clean_path(p)
    if not p:
        return ''
    path = Path(p)
    if path.is_dir():
        cand = path / ('saga_cmd.exe' if os.name == 'nt' else 'saga_cmd')
        return str(cand) if cand.exists() else ''
    return str(path) if path.exists() else ''

def _prepend_to_path(dirpath: Path):
    os.environ['PATH'] = str(dirpath) + os.pathsep + os.environ.get('PATH', '')

def _build_wind_effect_cmd(saga_exe, dem_path, out_base, dir_to_deg,
                           maxdist_km=DEFAULT_MAXDIST_KM,
                           accel=DEFAULT_ACCEL,
                           pyramids=DEFAULT_PYRAMIDS,
                           oldver=DEFAULT_OLDVER):
    out_sgrd = str(Path(out_base).with_suffix('.sgrd'))
    args = [
        saga_exe, 'ta_morphometry', '15',
        '-DEM', str(dem_path),
        '-DIR_UNITS', '1',                 # degrees
        '-EFFECT', out_sgrd,
        '-DIR_CONST', f'{dir_to_deg:.6f}',
        '-MAXDIST', f'{maxdist_km:.6f}',
        '-ACCEL', f'{accel:.6f}',
        '-PYRAMIDS', 'true' if pyramids else 'false',
        '-OLDVER', 'true' if oldver else 'false',
    ]
    return args, out_sgrd


def _build_export_tif_cmd(saga_exe, in_sgrd_path, out_tif_path):
    return [saga_exe, 'io_gdal', '2', '-GRIDS', str(in_sgrd_path), '-FILE', str(out_tif_path)]


def _run_quiet(cmd_list):
    """Run a command quietly. Return (exit_code, captured_tail) where captured_tail are last lines on error."""
    proc = subprocess.Popen(cmd_list, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    tail = []
    assert proc.stdout is not None
    for line in proc.stdout:
        tail.append(line.rstrip())
        if len(tail) > 40:
            tail.pop(0)
    proc.wait()
    return proc.returncode, "\n".join(tail)

# === Quick-look map helpers (matplotlib) ===
# We avoid specifying colors to keep things simple; default colormap will be used.

def _quicklook_png(raster_path: Path, out_png: Path, max_dim: int = 1024):
    import numpy as np
    import rasterio
    from rasterio.enums import Resampling
    import matplotlib.pyplot as plt

    raster_path = Path(raster_path)
    out_png = Path(out_png)
    out_png.parent.mkdir(parents=True, exist_ok=True)

    with rasterio.open(raster_path) as src:
        band1 = 1
        scale = max(src.width / max_dim, src.height / max_dim, 1)
        out_h = max(1, int(src.height / scale))
        out_w = max(1, int(src.width / scale))
        data = src.read(band1, out_shape=(out_h, out_w), resampling=Resampling.bilinear)
        if src.nodata is not None:
            mask = data == src.nodata
        else:
            mask = None

    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)
    if mask is not None:
        import numpy as np
        data = np.ma.array(data, mask=mask)
    ax.imshow(data)
    ax.axis('off')
    fig.tight_layout(pad=0)
    fig.savefig(out_png, dpi=150, bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return out_png

print('Config loaded. Go to Cell 2 to use the UI.')


In [ ]:
# --- Cell 2: UI + Run button (quiet) ---
out_log = widgets.Output(layout=widgets.Layout(border='1px solid #ddd', max_height='180px', overflow='auto'))
out_maps = widgets.Output(layout=widgets.Layout(border='1px solid #ddd'))
status = widgets.HTML('<span style="color:#555;">Idle.</span>')

w_saga = widgets.Text(
    value=_detect_saga_cmd(),
    description='saga_cmd:',
    placeholder='Paste full path or folder containing saga_cmd(.exe)',
    layout=widgets.Layout(width='70%')
)

w_mode = widgets.ToggleButtons(
    options=[('FROM (met)', 'from'), ('TO (SAGA)', 'to')],
    value='from',
    description='Dir mode:'
)

w_dir = widgets.IntSlider(
    value=140, min=0, max=359, step=1,
    description='Direction (°):', continuous_update=False
)

w_dem_path = widgets.Text(
    value='',
    description='DEM path:',
    placeholder='e.g., C:/data/malaga_2m.tif or /data/dem.tif',
    layout=widgets.Layout(width='70%')
)

w_dem_upload = widgets.FileUpload(
    accept='.tif,.tiff,.sgrd,.sdat,.sg-grd,.sg-grd-z',
    multiple=False,
    description='Upload DEM'
)

# ------ Output directory chooser (ipyfilechooser if available, else Text + Browse...) ------
try:
    _HAVE_FILECHOOSER = True
except Exception:
    _HAVE_FILECHOOSER = False

if _HAVE_FILECHOOSER:
    w_out_dir = FileChooser(str(Path.cwd()))
    w_out_dir.title = 'Output directory'
    w_out_dir.show_only_dirs = True
    w_out_dir.use_dir_icons = True
    out_dir_ui = w_out_dir
else:
    w_out_dir_text = widgets.Text(
        value='',
        description='Output dir:',
        placeholder='e.g., C:/work/outputs or ./results',
        layout=widgets.Layout(width='70%')
    )
    w_out_dir_btn = widgets.Button(description='Browse…', icon='folder-open')
    out_dir_ui = widgets.HBox([w_out_dir_text, w_out_dir_btn])

    def _browse_out_dir(_):
        try:
            import tkinter as tk
            from tkinter import filedialog
            root = tk.Tk()
            root.withdraw()
            sel = filedialog.askdirectory(initialdir=str(Path.cwd()))
            root.destroy()
            if sel:
                w_out_dir_text.value = sel
        except Exception as e:
            with out_log:
                print(f'(Folder dialog unavailable) {e}')
    w_out_dir_btn.on_click(_browse_out_dir)

def _get_out_dir_path() -> Path | None:
    if _HAVE_FILECHOOSER:
        p = getattr(w_out_dir, 'selected_path', None) or getattr(w_out_dir, 'default_path', '')
        p = (p or '').strip()
        return Path(p) if p else None
    else:
        p = getattr(w_out_dir_text, 'value', '').strip()
        return Path(os.path.expandvars(os.path.expanduser(p))) if p else None
# -------------------------------------------------------------------------------------------

w_out_base = widgets.Text(
    value='',
    description='Output base:',
    placeholder='If empty → <DEM>_WindEffect',
    layout=widgets.Layout(width='70%')
)

w_maxdist = widgets.FloatText(value=DEFAULT_MAXDIST_KM, description='MAXDIST (km):', layout=widgets.Layout(width='35%'))
w_accel   = widgets.FloatText(value=DEFAULT_ACCEL,     description='ACCEL:',         layout=widgets.Layout(width='35%'))
w_pyramids= widgets.Checkbox(value=DEFAULT_PYRAMIDS,   description='PYRAMIDS')
w_oldver  = widgets.Checkbox(value=DEFAULT_OLDVER,     description='OLDVER')

adv_box   = widgets.VBox([widgets.HBox([w_maxdist, w_accel]), widgets.HBox([w_pyramids, w_oldver])])
accordion = widgets.Accordion(children=[adv_box]); accordion.set_title(0, 'Advanced (optional)')

w_export_tif = widgets.Checkbox(value=DEFAULT_EXPORT_TIF, description='Also export GeoTIFF (.tif)')

btn_run = widgets.Button(description='Run Wind Effect', button_style='primary', icon='play')

# ===================== NEW: live direction arrow =====================
def _compute_dir_to() -> float:
    """Direction passed to SAGA (TO, degrees)."""
    d = float(w_dir.value) % 360.0
    if w_mode.value == 'from':
        d = (d + 180.0) % 360.0
    return d

def _dir_arrow_svg(angle_deg: float) -> str:
    # 0° points up (North), rotation clockwise — matches slider convention.
    return f'''
    <div style="display:flex;align-items:center;gap:8px;">
      <svg width="54" height="54" viewBox="0 0 100 100" style="flex:0 0 auto">
        <g transform="rotate({angle_deg:.1f} 50 50)">
          <line x1="50" y1="22" x2="50" y2="72" stroke="#333" stroke-width="8" stroke-linecap="round"/>
          <polygon points="50,10 64,34 36,34" fill="#333"/>
        </g>
        <circle cx="50" cy="50" r="46" fill="none" stroke="#ddd" stroke-width="2"/>
      </svg>
      <div style="min-width:100px;font-family:monospace;color:#555;">
        TO (SAGA): {angle_deg:.0f}°
      </div>
    </div>
    '''

w_dir_arrow = widgets.HTML(_dir_arrow_svg(_compute_dir_to()))

def _update_dir_arrow(*_):
    w_dir_arrow.value = _dir_arrow_svg(_compute_dir_to())

# Update on slider and mode changes
w_dir.observe(_update_dir_arrow, names='value')
w_mode.observe(_update_dir_arrow, names='value')
# ====================================================================

def _display_quicklooks(dem_path: Path, out_tif: Path | None):
    out_maps.clear_output()
    with out_maps:
        try:
            import tempfile, matplotlib.pyplot as plt
            tmpdir = Path(tempfile.gettempdir())
            dem_png = _quicklook_png(Path(dem_path), tmpdir / (Path(dem_path).stem + '_preview.png'))
            if out_tif and Path(out_tif).exists():
                out_png = _quicklook_png(Path(out_tif), tmpdir / (Path(out_tif).stem + '_preview.png'))
            else:
                out_png = None
            fig = plt.figure(figsize=(10,5))
            ax1 = fig.add_subplot(121); ax1.imshow(plt.imread(str(dem_png))); ax1.set_title('Input DEM (preview)'); ax1.axis('off')
            ax2 = fig.add_subplot(122)
            if out_png:
                ax2.imshow(plt.imread(str(out_png))); ax2.set_title('Output Wind Effect (preview)')
            else:
                ax2.text(0.5,0.5,'Output not created yet',ha='center',va='center'); ax2.set_title('Output Wind Effect (pending)')
            ax2.axis('off'); fig.tight_layout(); display(fig); plt.close(fig)
        except Exception as e:
            print(f'(Preview unavailable) {e}')

def on_run_clicked(_):
    btn_run.disabled = True
    out_log.clear_output()
    status.value = '<span style="color:#0078d7;">Processing…</span>'
    with out_log:
        # Resolve saga_cmd
        raw = w_saga.value
        saga_exe = _ensure_saga_exe(raw) or _ensure_saga_exe(_detect_saga_cmd())
        if not saga_exe:
            print('ERROR: saga_cmd not found. Paste either the full path to saga_cmd.exe or its folder.')
            status.value = '<span style="color:#c00;">Failed (saga_cmd not found)</span>'
            btn_run.disabled = False
            return
        _prepend_to_path(Path(saga_exe).parent)

        # Read selected output directory and base input early (used below)
        out_dir_sel = _get_out_dir_path()
        base_input  = w_out_base.value.strip()
        base_input_path = Path(os.path.expandvars(os.path.expanduser(base_input))) if base_input else None

        # DEM
        dem_path = None
        val = w_dem_upload.value
        if val:
            try:
                if isinstance(val, dict):
                    up_item = next(iter(val.values()))
                    name = up_item['metadata']['name']
                    data = up_item['content']
                else:
                    up = val[0]
                    name = getattr(up, 'name', 'uploaded_dem')
                    data = getattr(up, 'content', None)
                    if data is None and isinstance(up, dict):
                        data = up.get('content')

                # Save uploaded DEM into chosen output directory (or fallback)
                if out_dir_sel:
                    target_dir = out_dir_sel
                elif base_input_path and str(base_input_path.parent) not in ('.', ''):
                    target_dir = base_input_path.parent
                else:
                    target_dir = Path('.')
                target_dir.mkdir(parents=True, exist_ok=True)

                dem_path = target_dir / name
                with open(dem_path, 'wb') as f:
                    f.write(data)
            except Exception as e:
                print(f'ERROR: Could not read uploaded file: {e}')
                status.value = '<span style="color:#c00;">Failed (upload)</span>'
                btn_run.disabled = False
                return
        else:
            p = w_dem_path.value.strip()
            if p:
                dem_path = Path(os.path.expandvars(os.path.expanduser(p)))

        if not dem_path or not dem_path.exists():
            print(f'ERROR: DEM not found: {dem_path}')
            status.value = '<span style="color:#c00;">Failed (DEM not found)</span>'
            btn_run.disabled = False
            return

        # Input preview
        _display_quicklooks(dem_path, None)

        # Direction (used by SAGA)
        dir_to = _compute_dir_to()

        # Compute output base using (1) chosen output dir, (2) optional base path/name, else (3) DEM name
        if base_input:
            base_name = base_input_path.stem
        else:
            base_name = f"{Path(dem_path).with_suffix('').name}_WindEffect"

        if out_dir_sel:
            base_folder = out_dir_sel
        elif base_input and str(base_input_path.parent) not in ('.', ''):
            base_folder = base_input_path.parent
        else:
            base_folder = Path(dem_path).parent

        base_folder.mkdir(parents=True, exist_ok=True)
        out_base = base_folder / base_name

        out_sgrd = out_base.with_suffix('.sgrd')
        out_tif  = out_base.with_suffix('.tif')

        # Run quietly
        cmd_we, out_sgrd_path = _build_wind_effect_cmd(
            saga_exe, dem_path, out_base, dir_to,
            maxdist_km=float(w_maxdist.value),
            accel=float(w_accel.value),
            pyramids=bool(w_pyramids.value),
            oldver=bool(w_oldver.value)
        )
        code, tail = _run_quiet(cmd_we)
        if code != 0:
            print('ERROR: saga_cmd failed (Wind Effect). Tail output:')
            print(tail)
            status.value = '<span style="color:#c00;">Failed</span>'
            btn_run.disabled = False
            return

        if bool(w_export_tif.value):
            cmd_tif = _build_export_tif_cmd(saga_exe, out_sgrd, out_tif)
            code, tail = _run_quiet(cmd_tif)
            if code != 0:
                print('WARNING: GeoTIFF export failed. Tail output:')
                print(tail)

        print(f'Finished. Outputs in: {base_folder}')
        status.value = '<span style="color:#2b8a3e;">Processed ✔</span>'
        _display_quicklooks(dem_path, out_tif if bool(w_export_tif.value) else None)

    btn_run.disabled = False

btn_run.on_click(on_run_clicked)

help_html = widgets.HTML("""
<p><b>How to use</b></p>
<ol>
<li>Paste the path to <code>saga_cmd.exe</code> (or the folder containing it).</li>
<li>Select wind direction and whether it's <i>FROM</i> (met) or <i>TO</i> (SAGA).</li>
<li>Provide a DEM via <b>path</b> or <b>Upload DEM</b>.</li>
<li>(Optional) Choose an <b>Output directory</b> (folder) and/or an <b>Output base</b> (name or full path).</li>
<li>Optionally set advanced params, then click <b>Run Wind Effect</b>.</li>
</ol>
""")

ui = widgets.VBox([
    status,
    w_saga,
    # Arrow sits right next to the mode + slider
    widgets.HBox([w_mode, w_dir, w_dir_arrow]),
    widgets.HBox([w_dem_path, w_dem_upload]),
    out_dir_ui,
    w_out_base,
    w_export_tif,
    accordion,
    btn_run,
    out_maps,
    out_log,
    help_html,
])

display(ui)
print('Ready.')

